In [1]:
import pandas as pd
import requests
import threading

from time import sleep
from os import path

In [2]:
auth_url = "https://accounts.spotify.com/api/token"
response = requests.post(auth_url, {
    "grant_type": "client_credentials",
    "client_id": "24432fee68ea464c94ed2eeaff451fea",
    "client_secret": "5a4e797fab984adb9375ee8c6b9fab95"
})
api_token = response.json()['access_token']
print("API Response Status:", response.status_code, "\nToken:", api_token)

API Response Status: 200 
Token: BQCXjIvfFxOHNU-qMVujLrusSVcC_5RtsGeOmiKG-aMjNi-X71_DkIXR3b-D7QGsrEVLs3F0tGMoINo65zh4LsaYDMIm__Kc2DXnxrm4F703Av0xhkk


In [3]:
data = pd.read_csv(path.expanduser("~") + path.sep + path.sep.join(["Onedrive", "Desktop", "tracks_features.csv"]))

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204025 entries, 0 to 1204024
Data columns (total 24 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1204025 non-null  object 
 1   name              1204022 non-null  object 
 2   album             1204014 non-null  object 
 3   album_id          1204025 non-null  object 
 4   artists           1204025 non-null  object 
 5   artist_ids        1204025 non-null  object 
 6   track_number      1204025 non-null  int64  
 7   disc_number       1204025 non-null  int64  
 8   explicit          1204025 non-null  bool   
 9   danceability      1204025 non-null  float64
 10  energy            1204025 non-null  float64
 11  key               1204025 non-null  int64  
 12  loudness          1204025 non-null  float64
 13  mode              1204025 non-null  int64  
 14  speechiness       1204025 non-null  float64
 15  acousticness      1204025 non-null  float64
 16  

In [5]:
data = data.drop(columns=["instrumentalness","liveness","time_signature","year","release_date","album_id","album","artist_ids","track_number","disc_number","explicit","danceability","acousticness","duration_ms","artists","mode","loudness","speechiness","valence"])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204025 entries, 0 to 1204024
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   id      1204025 non-null  object 
 1   name    1204022 non-null  object 
 2   energy  1204025 non-null  float64
 3   key     1204025 non-null  int64  
 4   tempo   1204025 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 45.9+ MB


In [7]:
data.head(10)

,id,name,energy,key,tempo
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,0.978,7,117.906
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,0.957,11,103.680
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,0.970,7,149.749
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,0.967,11,96.752
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,0.929,2,127.059
5,2LXPNLSMAauNJfnC58lSqY,Born of a Broken Man,0.848,2,148.282
6,3moeHk8eIajvUEzVocXukf,Born As Ghosts,0.976,9,90.395
7,4llunZfVXv3NvUzXVB3VVL,Maria,0.873,11,172.848
8,21Mq0NzFoVRvOmLTOnJjng,Voice of the Voiceless,0.882,7,83.371
9,6s2FgJbnnMwFTpWJZzvb6z,New Millennium Homes,0.861,9,92.777


In [8]:
url = "https://api.spotify.com/v1/audio-analysis/"
headers = {
    "Authorization": f"Bearer {api_token}"
}

num_tracks = len(data)
start = int(2 * (len(data)/3))
print(f"Number of tracks: {num_tracks}")
num_404, num_ok = 0, 0
new_data_filename = path.expanduser("~") + path.sep + path.sep.join(["Onedrive", "Desktop", "test.csv"])
with open(new_data_filename, "w", encoding="utf-8") as f: f.write("id,status,timbre")

print(f"Start: {start}")

Number of tracks: 1204025
Start: 802683


In [9]:
for i in range(start, num_tracks + 1):
    response = requests.get(f"{url}{data.iloc[i]["id"]}", headers=headers)
    num_404 += 1 if response.status_code != 200 else 0
    num_ok += 1 if response.status_code == 200 else 0
    print(f"\r[{i}/{num_tracks}] Status: {response.status_code} | OK: {num_ok} | 404: {num_404}" + " " * 15, end="")
    id = data["id"].iloc[i]
    status = response.status_code
    timbre = response.json()["segments"][0]["pitches"] if response.status_code == 200 else 0
    with open(new_data_filename, "a", encoding="utf-8") as f: f.write(f"\n{id},{status},\"{timbre}\"")

print(f"Number of tracks that returned a 404 status: {num_404}")

[802729/1204025] Status: 200 | OK: 46 | 404: 1               

KeyboardInterrupt: 